In [71]:
print("Hello ")

Hello 


In [72]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from scipy.stats import spearmanr

from sklearn.ensemble import RandomForestRegressor


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

In [73]:
def drop_extreme_outliers(df,threshold=3):
    for column in df.select_dtypes(include=['number']).columns:  # Only process numeric columns
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)] 
    return df

In [74]:
def split_dataset_by_country(df):
    df_fr = df[df['COUNTRY'] == 'FR']
    df_de = df[df['COUNTRY'] == 'DE']
    return df_fr, df_de

In [75]:
def drop_irrelevant_columns(df, country):
    if country == 'FR':
        columns_to_drop = ['COUNTRY', 'DE_CONSUMPTION', 'DE_FR_EXCHANGE', 'FR_NET_EXPORT',
                           'DE_NET_EXPORT', 'DE_NET_IMPORT', 'DE_GAS', 'DE_COAL', 'DE_HYDRO',
                           'DE_NUCLEAR', 'DE_SOLAR', 'DE_WINDPOW', 'DE_LIGNITE', 'DE_RESIDUAL_LOAD',
                           'DE_RAIN', 'DE_WIND', 'DE_TEMP']
    elif country == 'DE':
        columns_to_drop = ['COUNTRY', 'FR_CONSUMPTION', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT',
                           'FR_NET_EXPORT', 'FR_NET_IMPORT', 'FR_GAS', 'FR_COAL', 'FR_HYDRO',
                           'FR_NUCLEAR', 'FR_SOLAR', 'FR_WINDPOW', 'FR_RESIDUAL_LOAD',
                           'FR_RAIN', 'FR_WIND', 'FR_TEMP']
        
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]

    return df.drop(columns=columns_to_drop)

In [76]:
def fill_missing_values(df):
    return df.fillna(0)

In [77]:
X_df = pd.read_csv("C:/Users/Dell/Downloads/X_train_NHkHMNU.csv")
y_df = pd.read_csv("C:/Users/Dell/Downloads/y_train_ZAN5mwg.csv")
x_test = pd.read_csv("C:/Users/Dell/Downloads/X_test_final.csv")
df = pd.merge(X_df, y_df, on='ID')

In [78]:
df.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,TARGET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445,0.028313
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365,-0.112516
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952,-0.180840
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948,-0.260356
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378,-0.071733


In [79]:
'''
x_train.interpolate(method='polynomial',order=3, inplace=True)
x_train_clean = x_train.drop(['COUNTRY'], axis=1).fillna(0)

x_test.interpolate(method='polynomial',order=3, inplace=True)
x_test_clean = x_test.drop(['COUNTRY'], axis=1).fillna(0)

y_train_clean = y_train['TARGET']


missing_values_x_train_clean = x_train_clean.isnull().sum()
print(missing_values_x_train_clean)
print("========================================")

missing_values_y_train_clean = y_train_clean.isnull().sum()
print(missing_values_y_train_clean)


print("\nDataFrame Info:\n", x_train.info())

shape = x_train.shape
'''


'\nx_train.interpolate(method=\'polynomial\',order=3, inplace=True)\nx_train_clean = x_train.drop([\'COUNTRY\'], axis=1).fillna(0)\n\nx_test.interpolate(method=\'polynomial\',order=3, inplace=True)\nx_test_clean = x_test.drop([\'COUNTRY\'], axis=1).fillna(0)\n\ny_train_clean = y_train[\'TARGET\']\n\n\nmissing_values_x_train_clean = x_train_clean.isnull().sum()\nprint(missing_values_x_train_clean)\nprint("========================================")\n\nmissing_values_y_train_clean = y_train_clean.isnull().sum()\nprint(missing_values_y_train_clean)\n\n\nprint("\nDataFrame Info:\n", x_train.info())\n\nshape = x_train.shape\n'

In [80]:
# Fonction pour séparer les datasets par pays
def split_dataset_by_country(dataframe):
    df_fr = dataframe[dataframe['COUNTRY'] == 'FR']
    df_de = dataframe[dataframe['COUNTRY'] == 'DE']
    return df_fr, df_de

In [81]:
# Séparer les datasets par pays
df_fr, df_de = split_dataset_by_country(df)

In [82]:
df_de.head()

,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET,TARGET
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948,-0.260356
11,819,116,DE,-0.055692,-0.811337,0.237105,-0.237105,-0.851082,-1.091142,0.851082,...,1.608304,2.014508,1.143607,1.777992,1.595158,0.158463,-0.359866,-0.203952,-0.376234,-0.133381
12,918,406,DE,0.532116,-0.331101,0.339942,-0.339942,-0.173123,-1.312029,0.173123,...,-0.194927,-1.394500,1.383171,0.100498,1.241892,-0.206340,1.170760,0.133643,0.033874,0.196312
13,283,1175,DE,-0.328286,-1.062255,-1.380464,1.380464,-1.046122,1.002243,1.046122,...,0.974454,-0.069416,-0.138918,-0.695013,-0.634046,-0.168491,0.122818,0.220077,5.453331,-0.025477
14,158,309,DE,1.028987,1.629315,1.129663,-1.129663,-0.391261,-1.823117,0.391261,...,0.194458,-0.516738,-1.106067,-0.945562,-0.667496,-1.566773,0.689483,1.095473,0.342798,0.460278


In [83]:
df_de = fill_missing_values(df_de)

In [84]:
df_de = drop_irrelevant_columns(df_de, 'DE')

In [85]:
y_de = df_de.pop('TARGET')

In [86]:
scaler_de = StandardScaler()

In [87]:
df_de_scaled = pd.DataFrame(scaler_de.fit_transform(df_de), columns=df_de.columns)

In [88]:
X_de = df_de_scaled

In [89]:
print('a')

a


In [90]:
#Splitting the data into training and test sets
x_train_final, x_test_final, y_train_final, y_test_final = train_test_split(X_de, y_de, test_size=0.2, random_state=42)

#For model calibration
#X_train_array = x_train_final.values
#y_train_array = y_train_final.values

# Convert numpy arrays to PyTorch tensors
#X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train_array, dtype=torch.float32)
#y_train_tensor = y_train_tensor.unsqueeze(1)

In [91]:
#Convert dataframes to numpy arrays
X_train_array = X_de.values
y_train_array = y_de.values

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_array, dtype=torch.float32)
y_train_tensor = y_train_tensor.unsqueeze(1)

In [92]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [93]:
def spearman_corr(y_true, y_pred):
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
    corr, _ = spearmanr(y_true, y_pred)
    return corr

In [96]:
class SpearmanLoss(torch.nn.Module):
    def __init__(self):
        super(SpearmanLoss, self).__init__()

    def forward(self, y_true, y_pred):
        # Calculer la corrélation de Spearman
        corr = spearman_corr(y_true, y_pred)
        
        # Retourner la version négative de la corrélation (pour minimiser la perte)
        return -corr

In [118]:
# Shuffle data before splitting (optional but recommended)
# indices = np.random.permutation(len(x_train_clean))
# X_train_shuffled = x_train_clean.iloc[indices]  # Use iloc to access rows by integer location
# y_train_shuffled = x_train_clean.iloc[indices]

# Define hyperparameters
input_dim = X_de.shape[1]
hidden_dim = 64
output_dim = 1  # Number of output neurons (for regression)
learning_rate = 0.001
num_epochs = 50
batch_size = 32

# Initialize k-fold cross-validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True)

# Initialize lists to store evaluation metrics
spearman_scores = []

# Perform k-fold cross-validation
for fold, (train_indices, val_indices) in enumerate(kf.split(X_de)):
    if fold < 3:  # Ignore the first two folds
        continue
        
    print(f"Fold {fold -2}/{k_folds}")
    
    # Shuffle data before splitting
    indices = np.random.permutation(len(X_de))
    x_train_shuffled = X_de.iloc[indices]
    y_train_shuffled = y_de.iloc[indices]
    
    # Split data into training and validation sets
    X_train_fold, X_val_fold = x_train_shuffled.iloc[train_indices], x_train_shuffled.iloc[val_indices]
    y_train_fold, y_val_fold = y_train_shuffled.iloc[train_indices], y_train_shuffled.iloc[val_indices]
    
    # Convert data to compatible data types
    X_train_fold = X_train_fold.astype('float32')
    y_train_fold = y_train_fold.astype('float32')
    X_val_fold = X_val_fold.astype('float32')
    y_val_fold = y_val_fold.astype('float32')
    
    # Convert data to PyTorch tensors
    train_data = TensorDataset(torch.tensor(X_train_fold.values, dtype=torch.float32),
                               torch.tensor(y_train_fold.values, dtype=torch.float32))
    val_data = TensorDataset(torch.tensor(X_val_fold.values, dtype=torch.float32),
                             torch.tensor(y_val_fold.values, dtype=torch.float32))
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)

    # Initialize model
    model = FeedForwardNN(input_dim, hidden_dim, output_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1, 1))  # Reshape labels to match output dimension
            loss.backward()
            optimizer.step()
    
    # Evaluate model
    model.eval()
    spearman_total = 0.0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            spearman_total += spearman_corr(labels, outputs)  # Use Spearman correlation here
    
    # Average Spearman score for this fold
    avg_spearman = spearman_total / len(val_loader)

    spearman_scores.append(avg_spearman)
    print(f"Spearman correlation for fold {fold + 1}: {avg_spearman:.4f}")

# Calculate average Spearman correlation across all folds
average_spearman = sum(spearman_scores) / len(spearman_scores)
print(f"Average Spearman correlation: {average_spearman:.4f}")


Fold 1/10
Spearman correlation for fold 4: 0.1338
Fold 2/10
Spearman correlation for fold 5: 0.4872
Fold 3/10
Spearman correlation for fold 6: 0.3046
Fold 4/10
Spearman correlation for fold 7: 0.5134
Fold 5/10
Spearman correlation for fold 8: 0.2883
Fold 6/10
Spearman correlation for fold 9: 0.2203
Fold 7/10
Spearman correlation for fold 10: 0.3702
Average Spearman correlation: 0.3311


In [95]:
# Convert the test set to PyTorch tensor
X_test_tensor = torch.tensor(x_test_final.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_final.values, dtype=torch.float32)

# Convert data to PyTorch tensor
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader for the test set
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Evaluate model on test set
model.eval()
mse_test = 0.0
num_samples = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        # Reshape the output tensor if needed
        outputs = outputs.view(-1)  # Remove the extra dimension
        # Reshape the labels tensor to match the shape of the outputs
        labels = labels.view(-1)   # Ensure labels have the same shape as outputs
        mse_test += criterion(outputs, labels).item() * len(inputs)
        num_samples += len(inputs)
mse_test /= num_samples

# Calculate RMSE
rmse_test = np.sqrt(mse_test)

print(f"Out-of-Sample MSE: {mse_test}")
print(f"Out-of-Sample RMSE: {rmse_test}")

Out-of-Sample MSE: 0.7892698091129924
Out-of-Sample RMSE: 0.8884085823048944
